In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
import re
import joblib # 모델 저장을 위해
import warnings

from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CBF 모델

In [8]:
# 0. 설정
warnings.simplefilter(action='ignore', category=FutureWarning)
print("[CBF] 1. (Train) 데이터 로드")

# 1. 데이터 로드
path_cf_data = "/content/drive/MyDrive/ML_Term_Project_Team11/density_processed.csv"
path_meta_data = "/content/drive/MyDrive/ML_Term_Project_Team11/metacritic_light_content_metadata.csv"

try:
    cf_data = pd.read_csv(path_cf_data, index_col=0)
    meta_data = pd.read_csv(path_meta_data, on_bad_lines='skip')
    print("CBF: Train 데이터 로드 성공")
except Exception as e:
    print(f"오류: {e}")
    raise e

# 2.1: 데이터 정제 및 동기화
print("\n[CBF] 2.1: 데이터 정제 및 동기화 (MetaCritic 기준)")

# 문자열 정규화 함수
def clean_title_for_match(title):
    # 특수문자 제거 및 공백 정리
    cleaned = str(title).replace(u'\xa0', u' ').replace(u'\u200b', u' ').replace('\r', ' ').replace('\n', ' ').strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned

# 1. CF(Train) 데이터의 996개 게임 목록을 정규화
cf_data.columns = [clean_title_for_match(col) for col in cf_data.columns]
games_in_cf = cf_data.columns

# 2. CBF(Content) 메타데이터의 'Game_Title' 열을 정규화
meta_data_unique = meta_data.drop_duplicates(subset=['Game_Title'], keep='first').copy()
meta_data_unique['Game_Title'] = meta_data_unique['Game_Title'].apply(clean_title_for_match)

# 3. 'Game_Title'을 인덱스로 설정
meta_data_indexed = meta_data_unique.set_index('Game_Title')

# 4. CF의 996개 게임 목록(games_in_cf) 순서대로 메타데이터를 재정렬
meta_data_aligned = meta_data_indexed.reindex(games_in_cf).reset_index(names=['Game_Title'])

# 5. 메타데이터가 없는 게임(NaN)은 빈 문자열('')로 채우기
meta_data_aligned.fillna('', inplace=True)

missing_count = meta_data_aligned['Genres'].replace('', np.nan).isnull().sum()
print(f"CBF: {missing_count} / 996 게임이 콘텐츠 정보 없음 (정상)")

# 2.2: TF-IDF 벡터화 (장르 3:1 가중치 적용)
print("\n[CBF] 2.2: TF-IDF 및 코사인 유사도 계산")

# 'Genres'와 'Description'을 결합하여 콘텐츠 텍스트 생성 (장르에 3배 가중치 부여)
meta_data_aligned['content'] = (meta_data_aligned['Genres'] + ' ') * 3 + meta_data_aligned['Description']

# TF-IDF Vectorizer 초기화
tfidf = TfidfVectorizer(stop_words='english')

# TF-IDF 행렬 생성
tfidf_matrix = tfidf.fit_transform(meta_data_aligned['content'])

# 2.3: 코사인 유사도 계산
# TF-IDF 행렬을 사용하여 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"\n[CBF] 2.3: 코사인 유사도 행렬 생성 완료")

# 2.4: 하이브리드용 결과물 저장
print("\n[CBF] 2.4: 하이브리드용 결과물 저장")

# 1. 코사인 유사도 행렬 (NumPy 배열)
np.save('/content/drive/MyDrive/ML_Term_Project_Team11/results/cbf_cosine_sim.npy', cosine_sim)

# 2. 정렬된 메타데이터 (CSV 파일)
meta_data_aligned.to_csv('/content/drive/MyDrive/ML_Term_Project_Team11/results/cbf_meta_data_aligned.csv', index=False, encoding='utf-8-sig')

print("CBF 모델 구축 및 파일 저장 완료: cbf_cosine_sim.npy, cbf_meta_data_aligned.csv")

[CBF] 1. (Train) 데이터 로드
CBF: Train 데이터 로드 성공

[CBF] 2.1: 데이터 정제 및 동기화 (MetaCritic 기준)
CBF: 547 / 996 게임이 콘텐츠 정보 없음 (정상)

[CBF] 2.2: TF-IDF 및 코사인 유사도 계산

[CBF] 2.3: 코사인 유사도 행렬 생성 완료

[CBF] 2.4: 하이브리드용 결과물 저장
CBF 모델 구축 및 파일 저장 완료: cbf_cosine_sim.npy, cbf_meta_data_aligned.csv


CF 모델

In [9]:
# 0. 설정
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. CBF와 동일한 정규화 함수
# CBF와 CF 간 게임 제목 정규화 일관성 유지
def clean_title_for_match(title):
    # \xa0, \u200b 같은 유니코드 공백이나 개행문자, 앞뒤 공백 등을 제거
    cleaned = str(title).replace(u'\xa0', u' ').replace(u'\u200b', u' ').replace('\r', ' ').replace('\n', ' ').strip()
    # 연속된 공백(e.g., 'A  B')을 하나의 공백('A B')으로 통일
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned

print("[CF] 1. (Train) 데이터 로드")
path_cf_data = "/content/drive/MyDrive/ML_Term_Project_Team11/density_processed.csv"

try:
    cf_data = pd.read_csv(path_cf_data, index_col=0)
    print(f"CF: '{path_cf_data}' 로드 성공 (Shape: {cf_data.shape})")
except Exception as e:
    print(f"오류: {e}")
    raise e

# 2.1: 게임 목록(컬럼) 정규화
print("\n[CF] 2.1: 게임 목록 정규화 (CBF와 동기화)")
cf_data.columns = [clean_title_for_match(col) for col in cf_data.columns]
print("CF: 996개 게임 컬럼 정규화 완료")

# 2.2: SVD 모델 학습
print("\n[CF] 2.2: SVD 모델 학습 시작")

# SVD (특이값 분해)는 행렬 분해(Matrix Factorization) 기법 중 하나로, NaN(결측값)을 처리
cf_data_filled = cf_data.fillna(0)

# TruncatedSVD 모델을 초기화
svd = TruncatedSVD(n_components=100, random_state=42)

# 0으로 채워진 평점 행렬에 대해 SVD 모델 학습
svd.fit(cf_data_filled)
print("CF: SVD 모델 학습 완료")

# 2.3: 예측 평점 행렬 생성
print("\n[CF] 2.3: 예측 평점 행렬 생성 중")

# 1. 학습된 SVD 모델을 사용하여 예측 평점 행렬 생성
U = svd.transform(cf_data_filled)

# 2. SVD 모델에서 학습된 아이템-잠재요인 행렬과 사용자-잠재요인 행렬을 곱하여 예측 평점 행렬 생성
V_T = svd.components_

# 3. 두 행렬을 내적하여 예측 평점 행렬 생성
R_pred_matrix = np.dot(U, V_T)

# 4. 예측 평점 행렬을 DataFrame으로 변환
R_pred_df = pd.DataFrame(
    R_pred_matrix,
    index=cf_data.index,    # (446 평론가)
    columns=cf_data.columns # (996 정규화된 게임)
)
print("CF: 예측 평점 행렬 (446x996) 생성 완료")

# 2.4: 하이브리드용 결과물 저장
print("\n[CF] 2.4: 하이브리드용 결과물 저장 중")
R_pred_df.to_csv('/content/drive/MyDrive/ML_Term_Project_Team11/results/R_pred_df.csv', encoding='utf-8-sig')
joblib.dump(svd, '/content/drive/MyDrive/ML_Term_Project_Team11/results/svd_model.pkl')
print("CF 모델 구축 및 파일 저장 완료: R_pred_df.csv, svd_model.pkl")

[CF] 1. (Train) 데이터 로드
CF: '/content/drive/MyDrive/ML_Term_Project_Team11/density_processed.csv' 로드 성공 (Shape: (446, 996))

[CF] 2.1: 게임 목록 정규화 (CBF와 동기화)
CF: 996개 게임 컬럼 정규화 완료

[CF] 2.2: SVD 모델 학습 시작
CF: SVD 모델 학습 완료

[CF] 2.3: 예측 평점 행렬 생성 중
CF: 예측 평점 행렬 (446x996) 생성 완료

[CF] 2.4: 하이브리드용 결과물 저장 중
CF 모델 구축 및 파일 저장 완료: R_pred_df.csv, svd_model.pkl


하이브리드 결합

In [11]:
# 0. 설정 및 정규화 함수
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# CBF, CF와 동일한 정규화 함수
def clean_title_for_match(title):
    cleaned = str(title).replace(u'\xa0', u' ').replace(u'\u200b', u' ').replace('\r', ' ').replace('\n', ' ').strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned

print("[하이브리드] 1. CF/CBF 모델 결과물 로드")

try:
    # 1.1: CF 모델 로드
    R_pred_df = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/results/R_pred_df.csv", index_col=0)
    R_pred_df.columns = [clean_title_for_match(col) for col in R_pred_df.columns]
    print("하이브리드: CF 예측 평점(R_pred_df.csv) 로드 성공")

    # 1.2: CBF 모델 로드
    cosine_sim = np.load("/content/drive/MyDrive/ML_Term_Project_Team11/results/cbf_cosine_sim.npy")
    meta_data_aligned = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/results/cbf_meta_data_aligned.csv")
    print("하이브리드: CBF 유사도 행렬(cbf_cosine_sim.npy) 로드 성공")
    print("하이브리드: CBF 게임 목록(cbf_meta_data_aligned.csv) 로드 성공")

    # 1.3: 원본 평점 데이터 로드
    cf_data_original = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/density_processed.csv", index_col=0)
    cf_data_original.columns = [clean_title_for_match(col) for col in cf_data_original.columns]
    print("하이브리드: 원본 평점(density_processed.csv) 로드 성공")

except Exception as e:
    print(f"오류: 모델 파일 로드 실패: {e}")
    raise e

# 1.4: CF/CBF 목록 일치 여부 검증
if list(R_pred_df.columns) == list(meta_data_aligned['Game_Title']):
    print("검증 성공: CF(996)와 CBF(996) 게임 목록이 100% 일치합니다")
else:
    print("!!! 검증 실패: CF/CBF 게임 목록이 일치하지 않습니다 !!!")
    raise SystemExit("모델 동기화 실패")

# 1.5: 하이브리드용 전역 변수 설정
scaler = MinMaxScaler() # 점수 정규화기 (0~1)
cbf_indices = pd.Series(meta_data_aligned.index, index=meta_data_aligned['Game_Title'])
games_in_cf = cf_data_original.columns # 996개 정규화된 게임 이름

print("\n[하이브리드] 2. 하이브리드 추천 함수 정의")

def get_hybrid_recommendations(critic_name, alpha=0.5, top_n=10):
    """
    CF(SVD) 점수와 CBF(콘텐츠) 점수를 가중치(alpha)로 결합
    """

    # --- [A] CF 점수 가져오기 ---
    cf_scores = R_pred_df.loc[critic_name]

    # --- [B] CBF 점수 계산하기 ---
    original_ratings = cf_data_original.loc[critic_name]
    liked_games = original_ratings[original_ratings >= 80].index
    liked_indices = [cbf_indices[game] for game in liked_games if game in cbf_indices.index]

    cbf_scores = []
    if not liked_indices:
        cbf_scores = np.zeros(len(games_in_cf))
    else:
        for game_idx in range(len(games_in_cf)):
            # 547개 게임은 cosine_sim[game_idx]가 모두 0이므로, avg_sim도 0
            avg_sim = np.mean([cosine_sim[game_idx, liked_idx] for liked_idx in liked_indices])
            cbf_scores.append(avg_sim)

    # --- [C] 하이브리드 점수 결합 ---
    hybrid_df = pd.DataFrame({
        'Game_Title': games_in_cf,
        'CF_Score': cf_scores.values,
        'CBF_Score': cbf_scores
    })

    hybrid_df['CF_Norm'] = scaler.fit_transform(hybrid_df[['CF_Score']])
    hybrid_df['CBF_Norm'] = scaler.fit_transform(hybrid_df[['CBF_Score']])

    # (콘텐츠 없는 547개 게임은 CBF_Norm이 0이므로 자동 CF 100% 반영)
    hybrid_df['Hybrid_Score'] = (alpha * hybrid_df['CF_Norm']) + ((1 - alpha) * hybrid_df['CBF_Norm'])

    rated_games = original_ratings.dropna().index
    hybrid_df = hybrid_df[~hybrid_df['Game_Title'].isin(rated_games)]

    hybrid_df = hybrid_df.sort_values(by='Hybrid_Score', ascending=False)

    return hybrid_df.head(top_n)

# 3. 하이브리드 추천 테스트 (MetaCritic 비평가)
print("\n[하이브리드] 3. 모델 결합 테스트 (데모)")
test_critic = cf_data_original.index[0] # '1UP'
print(f"\n[{test_critic}] 평론가 대상 추천 결과 (Alpha=0.5, 균형):")
print(get_hybrid_recommendations(test_critic, alpha=0.5))

[하이브리드] 1. CF/CBF 모델 결과물 로드
하이브리드: CF 예측 평점(R_pred_df.csv) 로드 성공
하이브리드: CBF 유사도 행렬(cbf_cosine_sim.npy) 로드 성공
하이브리드: CBF 게임 목록(cbf_meta_data_aligned.csv) 로드 성공
하이브리드: 원본 평점(density_processed.csv) 로드 성공
검증 성공: CF(996)와 CBF(996) 게임 목록이 100% 일치합니다

[하이브리드] 2. 하이브리드 추천 함수 정의

[하이브리드] 3. 모델 결합 테스트 (데모)

[1UP] 평론가 대상 추천 결과 (Alpha=0.5, 균형):
                           Game_Title   CF_Score  CBF_Score   CF_Norm  \
126        Call of Duty: World at War  35.126978   0.067298  0.428908   
599            Rise of Nations (2003)  27.070987   0.066846  0.364533   
386                       Homeworld 2  17.160349   0.063704  0.285337   
711  Starcraft II: Heart of the Swarm   4.371051   0.071135  0.183138   
712  Starcraft II: Legacy of the Void   3.435333   0.071135  0.175660   
153       Command & Conquer: Generals  19.425632   0.059245  0.303439   
19           Age of Mythology: Retold   2.222883   0.069822  0.165972   
181                            DOOM 3  63.387596   0.030337  0.654739   
103     

모델 평가

In [13]:
# 0. 설정 및 정규화 함수
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

def clean_title_for_match(title):
    cleaned = str(title).replace(u'\xa0', u' ').replace(u'\u200b', u' ').replace('\r', ' ').replace('\n', ' ').strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned

print("[평가] 1. Train(MetaCritic) vs Test(Steam) 비교")

try:
    # 1. Train(MetaCritic) 모델 결과물 로드
    R_pred_df_train = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/results/R_pred_df.csv", index_col=0)
    R_pred_df_train.columns = [clean_title_for_match(col) for col in R_pred_df_train.columns]
    meta_data_train = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/results/cbf_meta_data_aligned.csv")
    print("평가: Train 모델(CF/CBF) 결과물 로드 성공")

    # 2. Test(Steam) 데이터 로드
    steam_ratings_test = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/normalized_preprocessed_steam_matrix.csv", index_col=0)
    steam_ratings_test.columns = [clean_title_for_match(col) for col in steam_ratings_test.columns]
    print("평가: Test 평점(normalized_...) 로드 성공")

    steam_reviews_test = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/review_data.csv")
    steam_meta_test = pd.read_csv("/content/drive/MyDrive/ML_Term_Project_Team11/game_metadata_processed.csv")
    print("평가: Test 콘텐츠(review_data, game_metadata) 로드 성공")

except Exception as e:
    print(f"오류: 파일 로드 실패: {e}")
    raise e

# 3. [Task 1] 도메인 갭(Gap) 평가 (공통 6개 게임)
print("\n[Task 1] 도메인 갭 평가 (Train vs Test 6개 공통 게임)")
train_games_set = set(meta_data_train['Game_Title'])
test_games_set = set(steam_ratings_test.columns)
overlapping_games = list(train_games_set.intersection(test_games_set))
print(f"겹치는(Test 가능한) 게임 수: {len(overlapping_games)}")
print(f"겹치는 게임 목록: {overlapping_games}")

if len(overlapping_games) > 0:
    critic_pred_scores = R_pred_df_train[overlapping_games].mean(axis=0)
    steam_actual_scores = steam_ratings_test[overlapping_games].mean(axis=0)

    evaluation_df = pd.DataFrame({
        'Game_Title': overlapping_games,
        'Critic_Prediction (Train)': critic_pred_scores.values,
        'Steam_Actual (Test)': steam_actual_scores.values
    }).dropna()

    print("[평가 1] 도메인 간 평균 점수 비교 (MetaCritic vs Steam)")
    print(evaluation_df)

    rmse = np.sqrt(mean_squared_error(evaluation_df['Critic_Prediction (Train)'], evaluation_df['Steam_Actual (Test)']))
    print(f"\n[평가 2] 도메인 간 RMSE: {rmse:.4f} (두 도메인의 '취향 차이')")
else:
    print("Task 1 평가 실패: 겹치는 게임이 0개입니다")


# 4. [Task 2] 콜드 스타트 추천 평가 (Test 데이터 3개 모두 사용)
print("\n[Task 2] Steam 콜드 스타트 추천 평가")
# 제안서의 "콜드 스타트 문제 완화"를 평가하는 핵심 Task

# 1. "Test(Steam) CBF 모델" 구축 (review_data.csv 기반 - 804개)
print("   'Test(Steam) CBF 모델' 구축 중 (review_data.csv 사용)")
steam_reviews_test['review_text'] = steam_reviews_test['review_text'].fillna('')
game_reviews = steam_reviews_test.groupby('AppID')['review_text'].apply(' '.join).reset_index()
cbf_data_test = pd.merge(game_reviews, steam_meta_test[['AppID', 'name']], on='AppID', how='left')
cbf_data_test['name'] = cbf_data_test['name'].fillna('Unknown Game').apply(clean_title_for_match)
cbf_data_test = cbf_data_test.drop_duplicates(subset=['name'], keep='first').copy()

tfidf_test = TfidfVectorizer(max_features=5000)
tfidf_matrix_test = tfidf_test.fit_transform(cbf_data_test['review_text'])
cosine_sim_test = cosine_similarity(tfidf_matrix_test, tfidf_matrix_test)
indices_test = pd.Series(cbf_data_test.index, index=cbf_data_test['name'])
print("   'Test(Steam) CBF 모델' (804x804) 구축 완료")

def get_steam_cbf_recommendations(title, top_n=5):
    """Test(Steam) CBF 모델 추천 함수"""
    cleaned_title = clean_title_for_match(title)
    if cleaned_title not in indices_test:
        return f"'{title}' 게임은 Steam 리뷰 목록(804개)에 없습니다"
    idx = indices_test[cleaned_title]
    sim_scores = cosine_sim_test[idx]
    sim_scores_list = [(i, score) for i, score in enumerate(sim_scores) if i != idx]
    sim_scores_list = sorted(sim_scores_list, key=lambda x: x[1], reverse=True)
    game_indices = [i[0] for i in sim_scores_list[:top_n]]
    return cbf_data_test['name'].iloc[game_indices]

# 2. 콜드 스타트 테스트
# 'BLACK SOULS'는 OpenCritic(Train) 목록에 없는 Steam(Test) 전용 게임
test_cold_start_game = 'BLACK SOULS'
print(f"\n   콜드 스타트 추천 테스트 ('{test_cold_start_game}')")
print("   (CF 점수는 0이지만, CBF 점수만으로 추천 가능)\n")
print(get_steam_cbf_recommendations(test_cold_start_game))

[평가] 1. Train(MetaCritic) vs Test(Steam) 비교
평가: Train 모델(CF/CBF) 결과물 로드 성공
평가: Test 평점(normalized_...) 로드 성공
평가: Test 콘텐츠(review_data, game_metadata) 로드 성공

[Task 1] 도메인 갭 평가 (Train vs Test 6개 공통 게임)
겹치는(Test 가능한) 게임 수: 4
겹치는 게임 목록: ['Hades II', 'Cronos: The New Dawn', 'Hollow Knight: Silksong', 'Dying Light: The Beast']
[평가 1] 도메인 간 평균 점수 비교 (MetaCritic vs Steam)
                Game_Title  Critic_Prediction (Train)  Steam_Actual (Test)
0                 Hades II                   8.874009            81.236789
1     Cronos: The New Dawn                   7.525993            74.415566
2  Hollow Knight: Silksong                   8.085301            72.041901
3   Dying Light: The Beast                  13.008733            69.003900

[평가 2] 도메인 간 RMSE: 65.0701 (두 도메인의 '취향 차이')

[Task 2] Steam 콜드 스타트 추천 평가
   'Test(Steam) CBF 모델' 구축 중 (review_data.csv 사용)
   'Test(Steam) CBF 모델' (804x804) 구축 완료

   콜드 스타트 추천 테스트 ('BLACK SOULS')
   (CF 점수는 0이지만, CBF 점수만으로 추천 가능)

235             SILENT HI